# Use Case Description
Root cause analysis in cybersecurity is crucial as it helps identify the underlying causes of security incidents, enabling organizations to implement effective measures to prevent future breaches. However, this process is often tedious and time-consuming, since it requires sifting through vast amounts of data, correlating events, and manually detecting patterns. AI models can accelerates this by automating data analysis, improving accuracy, and reducing the effort needed to pinpoint the root causes efficiently.


## Model used for this use case
Both Instruct Model and Reasoning Model would be suitable for this task. In this example, we used Reasoning Model.  Oops, it is not available to the public quite yet.  Let's use instruct instead.

## Configuration
Update these variables to match your SageMaker deployment:

In [ ]:
# Update these variables to match your deployment
endpoint_name = 'foundation-sec-8b-endpoint'
aws_region = 'us-east-1'

print(f"Configuration:")
print(f"Endpoint: {endpoint_name}")
print(f"Region: {aws_region}")

In [ ]:
import boto3
import json
import re
from IPython.display import display, Markdown

# Initialize SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=aws_region)

print(f"Connected to SageMaker endpoint: {endpoint_name}")

In [ ]:
# Generation arguments optimized for classification (short outputs)
generation_args = {
    "max_new_tokens": 10,  # Small since we only need class labels
    "temperature": None,   # Deterministic for consistent classification
    "repetition_penalty": 1.2,
    "do_sample": False,
    "use_cache": True,
}

print("Generation configuration for classification:")
for key, value in generation_args.items():
    print(f"  {key}: {value}")

In [ ]:
def invoke_endpoint(prompt, max_new_tokens=1024, temperature=0.3):
    """Invoke the SageMaker endpoint with the given prompt"""
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "do_sample": temperature > 0,
            "repetition_penalty": 1.2
        }
    }
    
    try:
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        result = json.loads(response['Body'].read().decode())
        
        # Handle different response formats from TGI
        if isinstance(result, list) and len(result) > 0:
            return result[0].get('generated_text', str(result))
        elif isinstance(result, dict):
            return result.get('generated_text', str(result))
        else:
            return str(result)
            
    except Exception as e:
        print(f"Error invoking endpoint: {str(e)}")
        return f"Error: {str(e)}"

# Test the endpoint connection
test_response = invoke_endpoint("Hello, can you help with root cause analysis specific to Cyber Security?", max_new_tokens=50)
print("Test Response:")
print(test_response)

## Security Logs Assessment

Here’s a short mock security logs and configuration of firewalls

In [5]:
context = """

Sample Security Log

| Timestamp           | Source IP     | Destination IP | Protocol | Action   | Reason                  | User           |
|---------------------|---------------|----------------|----------|----------|-------------------------|----------------|
| 2025-07-14 22:15:03 | 192.168.1.45  | 10.10.10.5     | TCP      | Denied   | Suspicious Port Scan    | N/A            |
| 2025-07-14 22:15:10 | 192.168.1.45  | 10.10.10.5     | TCP      | Denied   | Suspicious Port Scan    | N/A            |
| 2025-07-14 22:15:15 | 192.168.1.45  | 10.10.10.5     | TCP      | Denied   | Suspicious Port Scan    | N/A            |
| 2025-07-14 22:16:00 | 192.168.1.45  | 10.10.10.5     | TCP      | Allowed  | Established Connection  | user123        |
| 2025-07-14 22:16:05 | 192.168.1.45  | 10.10.10.5     | TCP      | Allowed  | Data Transfer           | user123        |
| 2025-07-14 22:17:00 | 192.168.1.45  | 10.10.10.5     | TCP      | Denied   | Suspicious Payload      | user123        |

Firewall rules

| Rule ID | Source IP | Destination IP | Protocol | Port Range | Action |
|---------|-----------|----------------|----------|------------|--------|
| 101     | Any       | 10.10.10.5     | TCP      | 1-65535    | Allow  |


"""

display(Markdown(context))



Sample Security Log

| Timestamp           | Source IP     | Destination IP | Protocol | Action   | Reason                  | User           |
|---------------------|---------------|----------------|----------|----------|-------------------------|----------------|
| 2025-07-14 22:15:03 | 192.168.1.45  | 10.10.10.5     | TCP      | Denied   | Suspicious Port Scan    | N/A            |
| 2025-07-14 22:15:10 | 192.168.1.45  | 10.10.10.5     | TCP      | Denied   | Suspicious Port Scan    | N/A            |
| 2025-07-14 22:15:15 | 192.168.1.45  | 10.10.10.5     | TCP      | Denied   | Suspicious Port Scan    | N/A            |
| 2025-07-14 22:16:00 | 192.168.1.45  | 10.10.10.5     | TCP      | Allowed  | Established Connection  | user123        |
| 2025-07-14 22:16:05 | 192.168.1.45  | 10.10.10.5     | TCP      | Allowed  | Data Transfer           | user123        |
| 2025-07-14 22:17:00 | 192.168.1.45  | 10.10.10.5     | TCP      | Denied   | Suspicious Payload      | user123        |

Firewall rules

| Rule ID | Source IP | Destination IP | Protocol | Port Range | Action |
|---------|-----------|----------------|----------|------------|--------|
| 101     | Any       | 10.10.10.5     | TCP      | 1-65535    | Allow  |




In [6]:
def make_prompt(context):
   return f"""You are a security expert in charge of root cause analysis of security incidents.
    
    Go over the context given, and analyze the root cause of the issue
    
    ## CONTEXT
    {context}
    
    Respond with:
    - What is the issue?
    - What is the root cause?
    - How to make modifications to prevent the issue?

    When displaying a table, ensure it is clearly presented using Markdown.
    """

In [7]:
response = invoke_endpoint(make_prompt(context))
display(Markdown(response))

## Issue Analysis

### What is the issue?

The logs indicate that there have been multiple attempts by `192.168.1.45` (source) to connect to `10.10.10.5` (destination). Initially, these connections were denied due to suspicious port scans (`TCP Deny for Suspicious Port Scan`). However, after some time, an established connection was allowed (`Allowed Established Connection`) followed by data transfer. But then another attempt was denied again because of a suspicious payload (`TCP Denied for Suspicious Payload`).

**Issue:** The firewall is inconsistently applying its rules, allowing unauthorized access despite previous denials based on port scanning.

### Root Cause

**Root Cause:** 
The firewall rule set allows all TCP traffic from any source IP to destination `10.10.10.5` on ports 1-65535. This broad rule overrides more specific detections like port scan detection or malicious payloads. Since the allow rule has higher precedence than deny rules for the same protocol/port range, once the initial connection is allowed (after the port scan), subsequent legitimate traffic isn't blocked even if it's part of a potential attack sequence. 

Additionally, the system might not be properly configured to track sessions beyond the initial packet inspection. Therefore, once a connection is established, further packets within that session aren’t being inspected as aggressively, leading to false negatives where malicious activity occurs post-initial allowance.

### Solution Recommendations

To address this issue effectively:

#### Modify Firewall Rules

1. **Implement Layered Defense**: Create separate rules for different stages of communication. For example, block initial port scans entirely rather than just flagging them. Use rate limiting to detect and stop repeated probes.

2. **Update Existing Rules**: Ensure that the "Allow" rule for `Any -> 10.10.10.5 TCP 1-65535` is either removed or made more restrictive. Instead, define specific exceptions for known services required, e.g., only allow certain ports (like HTTP/HTTPS, SSH, etc.) instead of the entire range.

3. **Anomaly Detection Integration**: Integrate with IDS/IPS systems that can perform deeper inspection and anomaly-based blocking. These tools can look for patterns indicative of attacks even after a connection is established.

4. **Session Tracking Improvements**: If possible, configure the firewall to maintain stateful tracking of active sessions. This ensures that each packet is checked against the existing session’s legitimacy, preventing malicious activities during ongoing communications.

Here's how updated firewall rules could look like:

```markdown
| Rule ID | Source IP | Destination IP | Protocol | Port Range | Action         | Notes                           |
|---------|-----------|----------------|----------|------------|---------------|-----------------------------|
| 102     | Any       | 10.10.10.5     | TCP      | 80,443     | Allow         | HTTPS & SSH                 |
| 103     | Any       | 10.10.10.5     | TCP      | 0-1023     | Block          | Disallow non-approved ports   |
| 104     | Any       | 10.10.10.5     | TCP      | 1024-65535  | Detect & Alert | Monitor for unusual ports     |
```

This approach enforces stricter controls at various layers, reducing the risk of inconsistent application of policies and improving overall network security posture.